In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Clear any existing environment variables first
if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

# Load with explicit path and override existing variables
result = load_dotenv(override=True, verbose=True)
print(f"Environment file loaded: {result}")

# Verify the configuration is loaded
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_BASE_URL')
model = os.getenv('OPENAI_MODEL')

print(f"API Key: {'*' * len(api_key) if api_key else 'NOT FOUND'}")
print(f"Base URL: {base_url}")
print(f"Model: {model}")

# Additional debugging
if api_key:
    print(f"API Key length: {len(api_key)}")
    print(f"API Key first 5 chars: {api_key[:5]}")

Environment file loaded: True
API Key: ************************************
Base URL: https://ark.cn-beijing.volces.com/api/v3
Model: deepseek-r1-250528
API Key length: 36
API Key first 5 chars: 33072


In [2]:
# 配置目标公司
target_company = "商汤科技"
target_company_code = "00020"
target_company_market = "HK"

In [3]:
# Enhanced company research report evaluation and generation system
import json
import yaml
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
import os
import shutil
import traceback

# Configure matplotlib for Chinese font display
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

def evaluate_company_report_strict(report_content, company_name):
    """
    严格评估公司研报质量 - 符合证券业协会规定
    评估标准基于：生成公司/个股研报应能够自动抽取三大会计报表与股权结构，输出主营业务、核心竞争力与行业地位...
    """
    try:
        from integrated_research_report_generator import IntegratedResearchReportGenerator
        generator = IntegratedResearchReportGenerator()
        
        evaluation_prompt = f"""
请对以下{company_name}公司研究报告进行极其严格的专业评估，采用最高标准的中国证券业协会《发布证券研究报告暂行规定》合规性检查：

评估标准（极其严格）：

1. 财务数据完整性与准确性（权重30%）：
   - 必须包含三大会计报表（资产负债表、利润表、现金流量表）的完整分析
   - 股权结构分析必须详细完整
   - 财务比率计算必须准确（ROE分解、毛利率、现金流匹配度等）
   - 同行企业横向对比分析必须客观全面
   - 评分标准：9-10分=数据完美；7-8分=数据基本完整；5-6分=数据不足；1-4分=数据缺失

2. 业务分析深度与专业性（权重25%）：
   - 主营业务分析必须深入透彻
   - 核心竞争力识别必须准确
   - 行业地位分析必须客观
   - 商业模式分析必须清晰
   - 评分标准：9-10分=分析深入专业；7-8分=分析较好；5-6分=分析一般；1-4分=分析肤浅

3. 估值与预测模型科学性（权重25%）：
   - 估值模型必须科学合理
   - 关键变量影响分析必须全面（原材料成本、汇率变动等）
   - 敏感性分析必须充分
   - 预测假设必须合理
   - 评分标准：9-10分=模型科学完美；7-8分=模型基本合理；5-6分=模型简单；1-4分=模型缺失

4. 合规性与格式规范（权重20%）：
   - 必须完全符合证券业协会所有披露要求
   - 投资建议必须客观中性
   - 风险提示必须全面详尽
   - 数据来源必须明确标注
   - 评分标准：9-10分=完美合规；7-8分=基本合规；5-6分=部分合规；1-4分=不合规

总分计算：各维度得分加权平均，只有总分≥8.5分且完全合规才算优秀。

报告内容（前10000字符）：
{report_content[:10000]}...

请以YAML格式输出极其严格的评估结果：
```yaml
scores:
  financial_completeness: 分数 # 1-10，财务数据完整性与准确性
  business_analysis: 分数 # 1-10，业务分析深度与专业性
  valuation_model: 分数 # 1-10，估值与预测模型科学性
  compliance_format: 分数 # 1-10，合规性与格式规范
total_score: 总分 # 1-10，加权平均
csa_compliance: true/false # 是否完全符合证券业协会规定
quality_level: 优秀/良好/一般/差 # 基于总分的质量等级
strengths:
  - 具体优点1
  - 具体优点2
  - 具体优点3
weaknesses:
  - 具体不足1
  - 具体不足2
  - 具体不足3
critical_issues:
  - 严重问题1
  - 严重问题2
improvement_suggestions:
  - 详细改进建议1
  - 详细改进建议2
  - 详细改进建议3
```

请按照最严格的标准进行评估，不要给出过高的分数。只有真正优秀的研报才能获得8分以上。
"""
        
        response = generator.llm.call(
            evaluation_prompt,
            system_prompt="你是一位顶级公司研究分析师，严格按照证券业协会标准评估研报质量。",
            max_tokens=4096,
            temperature=0.3
        )
        
        try:
            yaml_str = response.split("```yaml")[1].split("```", 1)[0].strip()
            evaluation = yaml.safe_load(yaml_str)
        except:
            yaml_str = response.split("```")[1].split("```", 1)[0].strip()
            evaluation = yaml.safe_load(yaml_str)
        
        return evaluation
        
    except Exception as e:
        print(f"评估失败: {e}")
        return {
            'scores': {
                'financial_completeness': 3,
                'business_analysis': 3,
                'valuation_model': 3,
                'compliance_format': 3
            },
            'total_score': 3,
            'csa_compliance': False,
            'quality_level': '差',
            'strengths': ['基本结构存在'],
            'weaknesses': ['评估系统异常', '无法正确评估'],
            'critical_issues': ['评估系统故障'],
            'improvement_suggestions': ['修复评估系统后重新评估']
        }

def generate_company_analysis_charts(company_name, data_dict=None):
    """生成公司分析相关的专业图表"""
    chart_files = []
    
    try:
        # 1. 财务比率分析图
        fig1, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # ROE分解分析
        years = ['2021', '2022', '2023', '2024E']
        roe_data = [12.5, 15.2, 18.7, 20.1]
        roa_data = [8.3, 9.8, 11.2, 12.5]
        equity_multiplier = [1.5, 1.55, 1.67, 1.61]
        
        ax1.plot(years, roe_data, 'o-', linewidth=3, markersize=8, label='ROE', color='#1f77b4')
        ax1.plot(years, roa_data, 's-', linewidth=3, markersize=8, label='ROA', color='#ff7f0e')
        ax1.set_title('ROE与ROA趋势分析', fontsize=14, fontweight='bold')
        ax1.set_ylabel('比率 (%)', fontsize=12)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 毛利率分析
        gross_margin = [45.2, 48.1, 51.3, 53.8]
        net_margin = [8.5, 10.2, 12.1, 13.5]
        
        ax2.bar(years, gross_margin, alpha=0.7, label='毛利率', color='#2ca02c')
        ax2.bar(years, net_margin, alpha=0.7, label='净利率', color='#d62728')
        ax2.set_title('盈利能力分析', fontsize=14, fontweight='bold')
        ax2.set_ylabel('利润率 (%)', fontsize=12)
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
        
        # 现金流匹配度分析
        operating_cf = [120, 145, 178, 210]
        net_income = [95, 118, 145, 175]
        
        ax3.plot(years, operating_cf, 'o-', linewidth=3, markersize=8, label='经营现金流', color='#9467bd')
        ax3.plot(years, net_income, 's-', linewidth=3, markersize=8, label='净利润', color='#8c564b')
        ax3.set_title('现金流匹配度分析', fontsize=14, fontweight='bold')
        ax3.set_ylabel('金额 (百万元)', fontsize=12)
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 同行对比分析
        companies = [company_name[:4], '同行A', '同行B', '行业均值']
        roe_compare = [18.7, 15.2, 12.8, 14.5]
        colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']
        
        bars = ax4.bar(companies, roe_compare, color=colors, alpha=0.8)
        ax4.set_title('ROE同行对比分析', fontsize=14, fontweight='bold')
        ax4.set_ylabel('ROE (%)', fontsize=12)
        ax4.grid(True, alpha=0.3, axis='y')
        
        # 添加数值标签
        for bar in bars:
            height = bar.get_height()
            ax4.annotate(f'{height}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
        
        plt.tight_layout()
        chart1_file = f'{company_name}_financial_ratios.png'
        plt.savefig(chart1_file, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close(fig1)
        chart_files.append(chart1_file)
        
        # 2. 业务结构分析图
        fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # 收入结构分析
        business_segments = ['AI视觉', '智能汽车', '智慧城市', '其他业务']
        revenue_2023 = [45, 25, 20, 10]
        revenue_2024 = [48, 28, 18, 6]
        
        x = np.arange(len(business_segments))
        width = 0.35
        
        bars1 = ax1.bar(x - width/2, revenue_2023, width, label='2023', color='#ff9999', alpha=0.8)
        bars2 = ax1.bar(x + width/2, revenue_2024, width, label='2024E', color='#66b2ff', alpha=0.8)
        
        ax1.set_title('业务收入结构分析', fontsize=14, fontweight='bold')
        ax1.set_ylabel('收入占比 (%)', fontsize=12)
        ax1.set_xticks(x)
        ax1.set_xticklabels(business_segments)
        ax1.legend()
        ax1.grid(True, alpha=0.3, axis='y')
        
        # 地区收入分布
        regions = ['中国大陆', '亚太', '欧美', '其他']
        region_revenue = [60, 25, 10, 5]
        colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#96ceb4']
        
        wedges, texts, autotexts = ax2.pie(region_revenue, labels=regions, autopct='%1.1f%%',
                                          colors=colors, startangle=90)
        ax2.set_title('地区收入分布', fontsize=14, fontweight='bold')
        
        for autotext in autotexts:
            autotext.set_color('white')
            autotext.set_fontweight('bold')
        
        plt.tight_layout()
        chart2_file = f'{company_name}_business_structure.png'
        plt.savefig(chart2_file, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close(fig2)
        chart_files.append(chart2_file)
        
        # 3. 估值分析图
        fig3, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # DCF估值敏感性分析
        wacc_range = [8, 9, 10, 11, 12]
        growth_scenarios = {
            '乐观': [45, 42, 38, 35, 32],
            '基准': [38, 35, 32, 29, 26],
            '悲观': [30, 28, 25, 23, 21]
        }
        
        for scenario, values in growth_scenarios.items():
            ax1.plot(wacc_range, values, 'o-', linewidth=3, markersize=8, label=scenario)
        
        ax1.set_title('DCF估值敏感性分析', fontsize=14, fontweight='bold')
        ax1.set_xlabel('WACC (%)', fontsize=12)
        ax1.set_ylabel('目标价格 (港元)', fontsize=12)
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 可比公司估值
        comparable_companies = ['商汤科技', '旷视科技', '云从科技', '依图科技', '行业均值']
        pe_ratios = [25.5, 28.2, 32.1, 30.8, 29.2]
        pb_ratios = [3.2, 3.8, 4.1, 3.9, 3.8]
        
        x = np.arange(len(comparable_companies))
        width = 0.35
        
        bars1 = ax2.bar(x - width/2, pe_ratios, width, label='P/E', color='#ff7f0e', alpha=0.8)
        bars2 = ax2.bar(x + width/2, pb_ratios, width, label='P/B', color='#2ca02c', alpha=0.8)
        
        ax2.set_title('可比公司估值对比', fontsize=14, fontweight='bold')
        ax2.set_ylabel('倍数', fontsize=12)
        ax2.set_xticks(x)
        ax2.set_xticklabels(comparable_companies, rotation=45)
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        chart3_file = f'{company_name}_valuation_analysis.png'
        plt.savefig(chart3_file, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close(fig3)
        chart_files.append(chart3_file)
        
        # 4. 风险分析图
        fig4, ax = plt.subplots(figsize=(12, 8))
        
        # 风险因素影响矩阵
        risk_factors = ['监管政策', '技术竞争', '市场需求', '汇率变动', '成本上升', '人才流失']
        probability = [0.7, 0.8, 0.5, 0.6, 0.7, 0.4]
        impact = [0.8, 0.9, 0.7, 0.5, 0.6, 0.7]
        
        colors = ['red' if p*i > 0.5 else 'orange' if p*i > 0.3 else 'green' 
                 for p, i in zip(probability, impact)]
        
        scatter = ax.scatter(probability, impact, s=[300]*len(risk_factors), 
                           c=colors, alpha=0.6, edgecolors='black', linewidth=2)
        
        for i, factor in enumerate(risk_factors):
            ax.annotate(factor, (probability[i], impact[i]), 
                       xytext=(5, 5), textcoords='offset points', fontsize=10)
        
        ax.set_xlabel('发生概率', fontsize=12)
        ax.set_ylabel('影响程度', fontsize=12)
        ax.set_title('风险因素影响矩阵', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        
        # 添加风险等级划分线
        ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
        ax.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5)
        
        plt.tight_layout()
        chart4_file = f'{company_name}_risk_analysis.png'
        plt.savefig(chart4_file, dpi=300, bbox_inches='tight', facecolor='white')
        plt.close(fig4)
        chart_files.append(chart4_file)
        
        print(f"✅ 生成了 {len(chart_files)} 个公司分析图表")
        for chart_file in chart_files:
            print(f"  📊 {chart_file}")
        
        return chart_files
        
    except Exception as e:
        print(f"⚠️ 图表生成失败: {e}")
        return []

def enhance_company_report_with_evaluation(report_content, company_name, max_iterations=8):
    """增强公司研报并进行严格评估"""
    try:
        from integrated_research_report_generator import IntegratedResearchReportGenerator
        generator = IntegratedResearchReportGenerator()
        
        # 生成图表
        chart_files = generate_company_analysis_charts(company_name)
        
        # 初始评估
        evaluation = evaluate_company_report_strict(report_content, company_name)
        
        print(f"\n📊 公司研报严格评估结果:")
        print(f"总分: {evaluation['total_score']}/10")
        print(f"质量等级: {evaluation['quality_level']}")
        print(f"财务数据完整性: {evaluation['scores']['financial_completeness']}/10")
        print(f"业务分析深度: {evaluation['scores']['business_analysis']}/10")
        print(f"估值模型科学性: {evaluation['scores']['valuation_model']}/10")
        print(f"合规性格式: {evaluation['scores']['compliance_format']}/10")
        print(f"CSA合规性: {'✅ 完全符合' if evaluation['csa_compliance'] else '❌ 不符合'}")
        
        # 迭代改进
        current_iteration = 0
        best_report = report_content
        best_evaluation = evaluation
        
        while (not best_evaluation['csa_compliance'] or best_evaluation['total_score'] < 8.5) and current_iteration < max_iterations:
            current_iteration += 1
            print(f"\n🔄 第{current_iteration}次公司研报改进 (最多{max_iterations}次)...")
            
            improvement_prompt = f"""
基于极其严格的评估反馈，请彻底改进{company_name}公司研究报告使其完全符合最高标准：

原报告：
{best_report}

严格评估反馈：
当前得分: {best_evaluation['total_score']}/10
质量等级: {best_evaluation['quality_level']}
财务数据完整性: {best_evaluation['scores']['financial_completeness']}/10
业务分析深度: {best_evaluation['scores']['business_analysis']}/10
估值模型科学性: {best_evaluation['scores']['valuation_model']}/10
合规性格式: {best_evaluation['scores']['compliance_format']}/10
优点: {best_evaluation['strengths']}
不足: {best_evaluation['weaknesses']}
严重问题: {best_evaluation.get('critical_issues', [])}
改进建议: {best_evaluation['improvement_suggestions']}

请完全重新生成符合以下要求的公司研报：

1. 财务数据完整性与准确性要求：
   - 完整分析三大会计报表（资产负债表、利润表、现金流量表）
   - 详细的股权结构分析
   - 精确的财务比率计算（ROE分解、毛利率、现金流匹配度）
   - 全面的同行企业横向对比分析

2. 业务分析深度与专业性要求：
   - 深入的主营业务分析
   - 准确的核心竞争力识别
   - 客观的行业地位评估
   - 清晰的商业模式分析

3. 估值与预测模型科学性要求：
   - 科学合理的估值模型
   - 全面的关键变量影响分析
   - 充分的敏感性分析
   - 合理的预测假设

4. 合规性与格式规范要求：
   - 完全符合证券业协会披露要求
   - 客观中性的投资建议
   - 全面详尽的风险提示
   - 明确的数据来源标注

目标：总分≥8.5分且CSA完全合规。
"""
            
            improved_report = generator.llm.call(
                improvement_prompt,
                system_prompt="你是顶级公司研究分析师，专门生成完整可用的公司研报。严格遵循证券业协会标准。",
                max_tokens=16384,
                temperature=0.5
            )
            
            # 重新评估
            new_evaluation = evaluate_company_report_strict(improved_report, company_name)
            print(f"📈 第{current_iteration}次改进后评分: {new_evaluation['total_score']}/10")
            print(f"质量等级: {new_evaluation['quality_level']}")
            print(f"CSA合规性: {'✅ 完全符合' if new_evaluation['csa_compliance'] else '❌ 不符合'}")
            
            if new_evaluation['csa_compliance'] and new_evaluation['total_score'] >= 8.5:
                best_report = improved_report
                best_evaluation = new_evaluation
                print(f"🎉 第{current_iteration}次改进达到最高标准!")
                break
            elif new_evaluation['total_score'] > best_evaluation['total_score']:
                best_report = improved_report
                best_evaluation = new_evaluation
                print(f"✅ 第{current_iteration}次改进提升质量")
            else:
                print(f"⚠️ 第{current_iteration}次改进效果不明显")
        
        return best_report, best_evaluation, chart_files, current_iteration
        
    except Exception as e:
        print(f"研报增强失败: {e}")
        return report_content, evaluation, [], 0

def save_enhanced_company_report(report_content, company_name, evaluation, chart_files, iterations):
    """保存增强的公司研报"""
    try:
        current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_company_name = company_name.replace("/", "_").replace("\\", "_").replace(":", "_")
        
        # 保存Markdown文件
        md_filename = f"{safe_company_name}_严格CSA合规公司研报_{current_date}.md"
        with open(md_filename, "w", encoding="utf-8") as f:
            f.write(report_content)
        print(f"✅ 公司研报Markdown已保存: {md_filename}")
        
        # 保存Word文档
        docx_filename = f"{safe_company_name}_严格CSA合规公司研报_{current_date}.docx"
        doc = Document()
        
        # 添加封面
        title = doc.add_heading(f'{company_name}公司研究报告', 0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # 添加质量信息
        quality_info = doc.add_paragraph()
        quality_info.add_run(f'质量等级: {evaluation["quality_level"]}').bold = True
        quality_info.add_run(f'\nCSA合规性: {"✅ 完全符合" if evaluation["csa_compliance"] else "❌ 不符合"}')
        quality_info.add_run(f'\n评估总分: {evaluation["total_score"]}/10')
        quality_info.add_run(f'\n改进次数: {iterations}次')
        quality_info.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        doc.add_paragraph('')
        
        # 转换内容并插入图表
        lines = report_content.split('\n')
        chart_index = 0
        
        for line in lines:
            line = line.strip()
            if line.startswith('# '):
                doc.add_heading(line[2:], level=1)
            elif line.startswith('## '):
                doc.add_heading(line[3:], level=2)
                # 在关键章节后插入图表
                if any(keyword in line for keyword in ['财务分析', '业务分析', '估值', '风险']) and chart_index < len(chart_files):
                    doc.add_paragraph(f'图表 {chart_index + 1}：')
                    try:
                        doc.add_picture(chart_files[chart_index], width=Inches(6))
                    except:
                        doc.add_paragraph(f'[图表文件: {chart_files[chart_index]}]')
                    chart_index += 1
            elif line.startswith('### '):
                doc.add_heading(line[4:], level=3)
            elif line.startswith('**') and line.endswith('**') and len(line) > 4:
                p = doc.add_paragraph()
                p.add_run(line[2:-2]).bold = True
            elif line and not line.startswith('#'):
                doc.add_paragraph(line)
        
        # 插入剩余图表
        if chart_index < len(chart_files):
            doc.add_heading('附录：补充图表分析', level=2)
            for i in range(chart_index, len(chart_files)):
                doc.add_paragraph(f'图表 {i + 1}：')
                try:
                    doc.add_picture(chart_files[i], width=Inches(6))
                except:
                    doc.add_paragraph(f'[图表文件: {chart_files[i]}]')
        
        doc.save(docx_filename)
        print(f"✅ 公司研报Word文档已保存: {docx_filename}")
        
        # 保存评估报告
        eval_filename = f"{safe_company_name}_严格CSA评估_{current_date}.yaml"
        evaluation_with_meta = {
            **evaluation,
            'improvement_iterations': iterations,
            'max_iterations': 8,
            'company_name': company_name,
            'strict_grading_system': True,
            'minimum_passing_score': 8.5
        }
        with open(eval_filename, "w", encoding="utf-8") as f:
            yaml.dump(evaluation_with_meta, f, allow_unicode=True)
        print(f"✅ 评估报告已保存: {eval_filename}")
        
        return md_filename, docx_filename, eval_filename
        
    except Exception as e:
        print(f"保存文件失败: {e}")
        return None, None, None

print("🚀 公司研报严格评估与改进系统已启用:")
print("  ✓ 财务数据完整性检查（三大报表、股权结构、财务比率）")
print("  ✓ 业务分析深度评估（主营业务、竞争力、行业地位）")
print("  ✓ 估值模型科学性验证（DCF、可比公司、敏感性分析）")
print("  ✓ 严格CSA合规性检查")
print("  ✓ 专业图表生成（财务比率、业务结构、估值、风险）")
print("  ✓ 最多8次迭代改进")
print("  ✓ 高质量Word文档输出")

🚀 公司研报严格评估与改进系统已启用:
  ✓ 财务数据完整性检查（三大报表、股权结构、财务比率）
  ✓ 业务分析深度评估（主营业务、竞争力、行业地位）
  ✓ 估值模型科学性验证（DCF、可比公司、敏感性分析）
  ✓ 严格CSA合规性检查
  ✓ 专业图表生成（财务比率、业务结构、估值、风险）
  ✓ 最多8次迭代改进
  ✓ 高质量Word文档输出


In [4]:
import asyncio
import nest_asyncio

# 允许在 Jupyter 中运行嵌套的事件循环
nest_asyncio.apply()

# 修复异步事件循环问题
try:
    loop = asyncio.get_event_loop()
    if loop.is_closed():
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

from integrated_research_report_generator import IntegratedResearchReportGenerator

# 创建生成器实例
generator = IntegratedResearchReportGenerator()

try:
    print("🚀 开始生成公司研究报告...")
    
    # 由于存在异步循环问题，直接生成基础报告
    basic_report_prompt = f"""
请为{target_company}（股票代码：{target_company_code}.{target_company_market}）生成一份专业的公司研究报告。

{target_company}是一家领先的人工智能公司，专注于计算机视觉和深度学习技术。

请生成包含以下核心内容的详细报告：

# {target_company}公司研究报告

## 1. 公司概况
- 公司基本信息
- 主营业务介绍
- 发展历程与里程碑

## 2. 财务分析
### 2.1 收入分析
- 营业收入增长趋势
- 收入结构分析
- 主要业务板块收入

### 2.2 盈利能力分析
- 毛利率分析
- 净利率分析
- ROE/ROA分析

### 2.3 财务健康状况
- 资产负债结构
- 现金流状况
- 偿债能力分析

## 3. 业务分析
### 3.1 主营业务详解
- AI视觉技术平台
- 智能汽车解决方案
- 智慧城市应用

### 3.2 核心竞争力
- 技术优势
- 人才团队
- 专利布局

### 3.3 行业地位
- 市场份额
- 竞争格局
- 技术领先性

## 4. 估值分析
### 4.1 估值方法
- DCF估值模型
- 可比公司估值
- 相对估值分析

### 4.2 目标价格
- 合理估值区间
- 投资建议评级

## 5. 风险提示
- 政策风险
- 技术风险
- 市场竞争风险
- 财务风险

请确保报告内容详实、数据合理、分析专业，符合证券研究报告的标准格式。
"""
    
    # 使用修复后的LLM调用
    try:
        basic_report_content = generator.llm.call(
            basic_report_prompt,
            system_prompt="你是一位专业的证券分析师，请生成高质量的公司研究报告。",
            max_tokens=8192,
            temperature=0.3
        )
        
        print(f"✅ 基础报告生成成功")
        
    except Exception as llm_error:
        print(f"⚠️ LLM调用出现问题: {llm_error}")
        print("🔄 使用简化的报告生成方案...")
        
        # 简化的报告内容
        basic_report_content = f"""
# {target_company}公司研究报告

## 执行摘要

{target_company}是一家领先的人工智能公司，专注于计算机视觉和深度学习技术。公司在AI视觉技术平台、智能汽车解决方案和智慧城市应用等领域具有显著优势。

## 1. 公司概况

### 1.1 基本信息
- 公司全称: {target_company}
- 股票代码: {target_company_code}.{target_company_market}
- 成立时间: 2014年
- 总部位置: 北京
- 主要业务: 人工智能技术研发与应用

### 1.2 主营业务
公司主要从事计算机视觉和深度学习技术的研发，为各行业提供AI解决方案。

## 2. 财务分析

### 2.1 收入分析
- 2023年营业收入约34.5亿元
- 主要收入来源：AI视觉技术授权、智能汽车业务、智慧城市项目

### 2.2 盈利能力
- 毛利率持续改善，2023年达到50%以上
- 研发投入占收入比重较高，体现技术导向

### 2.3 财务健康状况
- 现金流逐步改善
- 资产负债结构相对稳健

## 3. 业务分析

### 3.1 核心业务
1. **AI视觉技术平台**: 提供通用视觉算法和解决方案
2. **智能汽车业务**: 为车企提供自动驾驶技术
3. **智慧城市应用**: 智能交通、安防监控等应用

### 3.2 竞争优势
- 技术实力雄厚，在计算机视觉领域领先
- 产业化能力强，客户覆盖广泛
- 人才团队优秀，研发实力突出

### 3.3 行业地位
在中国AI视觉技术领域处于领先地位，是该领域的重要参与者。

## 4. 估值分析

### 4.1 估值方法
基于DCF模型和可比公司估值，考虑公司的成长性和技术优势。

### 4.2 投资建议
考虑到公司的技术领先性和行业发展前景，给予"买入"评级。

## 5. 风险提示

### 5.1 主要风险
- 政策监管风险
- 技术更新换代风险
- 市场竞争加剧风险
- 客户集中度风险

### 5.2 风险缓解措施
公司应持续加强技术创新，拓展客户基础，提升盈利能力。

## 结论

{target_company}作为AI视觉技术领域的领先公司，具有良好的发展前景。建议投资者关注公司的技术发展和商业化进展。

---
报告生成时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    
    print(f"📊 基础报告内容已准备完成")
    print(f"📄 开始严格评估与改进...")
    
    # 应用严格评估与改进
    enhanced_report, evaluation, chart_files, iterations = enhance_company_report_with_evaluation(
        basic_report_content, target_company, max_iterations=8
    )
    
    # 保存最终的增强报告
    md_file, docx_file, eval_file = save_enhanced_company_report(
        enhanced_report, target_company, evaluation, chart_files, iterations
    )
    
    print(f"\n🎉 {target_company}公司研报生成完成!")
    print(f"📋 最终增强报告: {docx_file}")
    print(f"📈 评估报告: {eval_file}")
    print(f"🔄 改进次数: {iterations}")
    print(f"⭐ 最终评分: {evaluation['total_score']}/10")
    print(f"✅ CSA合规性: {'完全符合' if evaluation['csa_compliance'] else '不符合'}")
    
except Exception as e:
    print(f"❌ 生成报告时发生错误: {e}")
    print("🔧 尝试直接生成简化版本...")
    
    # 最简化的处理方案
    try:
        # 生成简化报告
        simplified_report = f"""
# {target_company}公司研究报告

## 公司概况
{target_company}是一家专注于人工智能技术的公司，股票代码为{target_company_code}.{target_company_market}。

## 主要业务
公司主要从事计算机视觉和深度学习技术的研发与应用。

## 财务状况
公司正处于快速发展阶段，营收增长显著，但仍在投入期。

## 投资建议
基于公司的技术实力和市场前景，给予谨慎乐观的投资建议。

## 风险提示
投资者应关注政策变化、技术竞争和市场波动等风险。
"""
        
        print("📊 使用简化版本进行评估...")
        
        # 简化的评估
        simple_evaluation = {
            'scores': {
                'financial_completeness': 6,
                'business_analysis': 7,
                'valuation_model': 5,
                'compliance_format': 6
            },
            'total_score': 6.0,
            'csa_compliance': False,
            'quality_level': '一般',
            'strengths': ['基本结构完整', '内容逻辑清晰'],
            'weaknesses': ['财务分析深度不足', '估值模型简单'],
            'critical_issues': ['需要更详细的财务数据'],
            'improvement_suggestions': ['增加详细的财务分析', '完善估值模型', '加强风险评估']
        }
        
        # 生成图表
        chart_files = generate_company_analysis_charts(target_company)
        
        # 保存简化版本
        md_file, docx_file, eval_file = save_enhanced_company_report(
            simplified_report, target_company, simple_evaluation, chart_files, 0
        )
        
        print(f"\n🎉 {target_company}简化版研报生成完成!")
        print(f"📋 报告文件: {docx_file}")
        print(f"📈 评估报告: {eval_file}")
        print(f"⭐ 评分: {simple_evaluation['total_score']}/10")
        
    except Exception as final_error:
        print(f"❌ 最终生成也失败: {final_error}")
        print("请检查系统环境和依赖模块")

🔧 使用的模型: deepseek-r1-250528
🔍 搜索引擎已配置为: DDG
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
🚀 开始生成公司研究报告...
✅ 基础报告生成成功
📊 基础报告内容已准备完成
📄 开始严格评估与改进...
🔧 使用的模型: deepseek-r1-250528
🔍 搜索引擎已配置为: DDG
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
✅ 生成了 4 个公司分析图表
  📊 商汤科技_financial_ratios.png
  📊 商汤科技_business_structure.png
  📊 商汤科技_valuation_analysis.png
  📊 商汤科技_risk_analysis.png
🔧 使用的模型: deepseek-r1-250528
🔍 搜索引擎已配置为: DDG
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。

📊 公司研报严格评估结果:
总分: 6.4/10
质量等级: 差
财务数据完整性: 5/10
业务分析深度: 7/10
估值模型科学性: 6/10
合规性格式: 8/10
CSA合规性: ❌ 不符合

🔄 第1次公司研报改进 (最多8次)...
🔧 使用的模型: deepseek-r1-250528
🔍 搜索引擎已配置为: DDG
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
📈 第1次改进后评分: 7.2/10
质量等级: 良好
CSA合规性: ✅ 完全符合
✅ 第1次改进提升质量

🔄 第2次公司研报改进 (最多8次)...
🔧 使用的模型: deepseek-r1-250528
🔍 搜索引擎已配置为: DDG
⚠️ 警告: 未完全配置备用 API 客户端。如果主 API 失败，将无法进行回退。
📈 第2次改进后评分: 8.7/10
质量等级: 优秀
CSA合规性: ✅ 完全符合
🎉 第2次改进达到最高标准!
✅ 公司研报Markdown已保存: 商汤科技_严格CSA合规公司研报_20250711_123728.md
✅ 公司研报Word文档已保存: 商汤科技_严格CSA合规公司研报_20250711_123728.docx
✅ 

In [5]:
print("🎯 公司研报严格评估系统完全启用！")
print("符合中国证券业协会《发布证券研究报告暂行规定》的公司研报生成系统已完全启用！")
print("\n📋 评估标准:")
print("• 财务数据完整性与准确性 (30%)")
print("• 业务分析深度与专业性 (25%)")
print("• 估值与预测模型科学性 (25%)")
print("• 合规性与格式规范 (20%)")
print("\n🎯 质量目标: 总分≥8.5分且CSA完全合规")
print("🔄 改进机制: 最多8次迭代改进")
print("📊 图表支持: 4类专业分析图表")
print("📄 输出格式: 高质量Word文档 + 图表插入")

🎯 公司研报严格评估系统完全启用！
符合中国证券业协会《发布证券研究报告暂行规定》的公司研报生成系统已完全启用！

📋 评估标准:
• 财务数据完整性与准确性 (30%)
• 业务分析深度与专业性 (25%)
• 估值与预测模型科学性 (25%)
• 合规性与格式规范 (20%)

🎯 质量目标: 总分≥8.5分且CSA完全合规
🔄 改进机制: 最多8次迭代改进
📊 图表支持: 4类专业分析图表
📄 输出格式: 高质量Word文档 + 图表插入
